In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

多候选者 vs 冠军方法论，用于模型部署到生产环境中

*注意：此笔记本使用的是KFP 1.x和GCPC 1.x。我们建议使用2.x版本*

注意：此笔记本已在以下环境中测试：

* Python版本=3.9

## 概述

本教程展示了如何使用Vertex AI Pipeline，通过多候选者对决冠军方法将下一个版本的模型部署到生产环境中。

### 目标

在本教程中，您将学习如何构建一个 Vertex AI 管道，该管道评估部署（生产）模型的新生产数据与其它版本（竞争者）的模型，以确定哪个竞争者模型将成为替换生产环境中的冠军模型。

本教程使用以下Google Cloud ML 服务和资源：

- Vertex AI Pipeline
- Vertex AI Model Evaluation
- Vertex AI Model Registry
- Vertex AI Endpoints

执行的步骤包括：

- 导入一个经过预训练的（冠军）模型到 `Vertex AI Model Registry`。
- 将合成模型训练评估指标导入对应的（冠军）模型。
- 创建一个 `Vertex AI Endpoint` 资源。
- 将冠军模型部署到 `Endpoint` 资源。
- 导入其他（竞争者）版本的部署模型。
- 将合成模型训练评估指标导入对应的（竞争者）模型。
- 创建一个 Vertex AI Pipeline
    - 获取冠军模型。
    - 使用生产数据（伪造）微调冠军模型。
    - 为冠军模型导入合成的训练+生产评估指标。
    - 获取竞争者模型。
    - 使用生产数据（伪造）微调竞争者模型。
    - 为竞争者模型导入合成的训练+生产评估指标。
    - 将竞争者的评估结果与冠军进行比较，并将新的冠军模型设置为默认模型。
    - 部署新的冠军模型。

了解有关 [Vertex AI 管道](https://cloud.google.com/vertex-ai/docs/pipelines/introduction) 的更多信息。

### 模型

本教程使用了来自TensorFlow Hub的预训练图像分类模型，该模型是在ImageNet数据集上训练的。

了解更多关于[ResNet V2预训练模型](https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5)。

### 费用

本教程使用谷歌云的可计费组件：

* Vertex AI
* 云存储

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)，
以及[云存储价格](https://cloud.google.com/storage/pricing)，
并使用[定价计算器](https://cloud.google.com/products/calculator/) 
根据您的预计使用量生成成本估算。

## 安装

安装以下所需的包以执行此笔记本。

In [ ]:
# Install the packages
! pip3 install --quiet --upgrade google-cloud-aiplatform \
                                 'google-cloud-pipeline-components<2'

! pip3 install --quiet           tensorflow==2.5 \
                                 tensorflow_hub

! pip3 install --quiet --upgrade 'kfp<2'

### 仅限协作：取消以下单元格的注释以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置你的Google Cloud项目

**以下步骤是必需的，无论你使用什么笔记本环境。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当你第一次创建一个帐户时，你会获得$300的免费信用额度，可用于支付计算/存储成本。

2. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API]。

4. 如果你正在本地运行这个笔记本，你需要安装[Cloud SDK](https://cloud.google.com/sdk)。

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[定位项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Region

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解更多关于[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

###验证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明进行操作。

1. 顶点 AI 工作台
* 无需操作，因为您已经通过身份验证。

2. 本地 JupyterLab 实例，取消注释并运行:

In [ ]:
# ! gcloud auth login

3. 使用协作编辑，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账户或其他
*请参考如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples上为您的服务账户授予云存储权限。

### 创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶不存在时，才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### 服务账号

您可以使用服务账号来创建Vertex AI管道作业。

如果您不想使用项目的计算引擎服务账号，请将`SERVICE_ACCOUNT`设置为另一个服务账号ID。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### 为Vertex AI Pipelines设置服务账户访问权限

运行以下命令，授予您的服务账户读取和写入管道存储桶中的管道工件的访问权限，此存储桶是在上一步中创建的 -- 您只需要针对每个服务账户运行一次这些命令。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 导入库

In [ ]:
import kfp
import tensorflow as tf
import tensorflow_hub as hub
from google.cloud import aiplatform
from google.cloud.aiplatform import gapic
from kfp.v2 import compiler
from kfp.v2.dsl import component

### 初始化 Vertex AI SDK for Python

为您的项目初始化 Vertex AI SDK for Python。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量 `DEPLOY_GPU/DEPLOY_NGPU` 来使用支持 GPU 的容器镜像，以及分配给虚拟机（VM）实例的 GPU 的数量。例如，要使用一个 GPU 容器镜像，并为每台 VM 分配 4 个 Nvidia Telsa K80 GPU，您可以指定：

    (aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定 `(None, None)` 来使用一个容器镜像在 CPU 上运行。

了解更多关于[您所在地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

In [ ]:
DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

为训练和预测设置预构建的Docker容器映像。

要查看最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TF = "2.5".replace(".", "-")

if DEPLOY_GPU:
    DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
else:
    DEPLOY_VERSION = "tf2-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量 `DEPLOY_COMPUTE` 设置为配置用于训练的VM的计算资源。
 - `机器类型`
     - `n1-standard`: 每个vCPU 3.75GB内存。
     - `n1-highmem`: 每个vCPU 6.5GB内存
     - `n1-highcpu`: 每个vCPU 0.9GB内存
 - `vCPUs`: \[2、4、8、16、32、64、96\]个CPU核心

*注意：以下内容不支持训练：*

 - `standard`: 2个vCPU
 - `highcpu`: 2、4和8个vCPU

*注意：您也可以在训练和部署中使用n2和e2机器类型，但它们不支持GPU*。

### 保存模型文件

在这一点上，模型存储在内存中。 接下来，您需要将模型文件保存到云存储位置。

In [ ]:
DEPLOY_COMPUTE = "n1-standard-4"
print("Deploy machine type", DEPLOY_COMPUTE)

从TensorFlow Hub中获取预训练模型

为了演示目的，本教程使用了来自TensorFlow Hub（TFHub）的预训练模型，然后上传到“Vertex AI Model”资源中。一旦有了“Vertex AI Model”资源，该模型就可以部署到“Vertex AI Endpoint”资源中。

### 下载预训练模型

首先，您可以从TensorFlow Hub中下载预训练模型。该模型作为TF.Keras层下载。在这个例子中，为了完成模型，创建一个带有下载的TFHub模型作为层的`Sequential()`模型，并指定模型的输入形状。

In [ ]:
tfhub_model = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model.build([None, 32, 32, 3])

tfhub_model.summary()

In [ ]:
MODEL_DIR = BUCKET_URI + "/model"
tfhub_model.save(MODEL_DIR)

### 将TensorFlow Hub模型上传到`Vertex AI Model`资源

最后，您可以使用`upload()`方法将TFHub模型的模型构件上传到`Vertex AI Model`资源，需要指定以下参数：

- `display_name`: `Model`资源的可读名称。
- `artifact_uri`: 模型包的存储位置。
- `serving_container_image_uri`: 提供服务的容器镜像。

将模型上传到Vertex AI Model资源会返回一个长时间运行的操作，因为可能需要一些时间。

*注意:* 当将模型构件上传到`Vertex AI Model`资源时，需要指定相应的部署容器镜像。

In [ ]:
champion_model = aiplatform.Model.upload(
    display_name="resnet-v1",
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    is_default_version=True,
    version_aliases=["v1"],
)

print(champion_model)

### 创建模型评估

首先，您以与模型评估的预定义模式之一相对应的格式创建模型评估。在这个例子中，您将使用分类指标的模式，并将以下评估指标子集指定为一个字典：

- `logLoss`：对数损失。
- `auPrc`：准确度。

然后，您使用以下参数构建 `ModelEvaluation` 对象：

- `display_name`：评估指标的人类可读名称。
- `metrics_schema_uri`：特定类型评估指标的模式。
- `metrics`：包含评估指标的字典。

了解更多关于[评估指标模式](https://cloud.google.com/vertex-ai/docs/evaluation/introduction#features)。

In [ ]:
metrics = {"logLoss": 1.4, "auPrc": 0.85}
print(metrics)

champion_eval = gapic.ModelEvaluation(
    display_name="train-v1",
    metrics_schema_uri="gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml",
    metrics=metrics,
)

### 将评估指标上传到模型注册表

接下来，将自定义训练作业中模型的评估上传到 Vertex AI 模型注册表中相应的条目。

目前，SDK 中尚不支持此方法。相反，您可以使用较低级别的 GAPIC API 接口。

In [ ]:
API_ENDPOINT = f"{REGION}-aiplatform.googleapis.com"
client = gapic.ModelServiceClient(client_options={"api_endpoint": API_ENDPOINT})

client.import_model_evaluation(
    parent=champion_model.resource_name, model_evaluation=champion_eval
)

### 创建`Endpoint`资源

您可以使用`Endpoint.create()`方法创建`Endpoint`资源。至少，您需要指定端点的显示名称。另外，您可以指定项目和位置（区域）；否则，设置将继承您使用`init()`方法初始化Vertex AI SDK时设置的值。

在这个例子中，指定了以下参数：

- `display_name`：`Endpoint`资源的可读名称。
- `project`：您的项目ID。
- `location`：您的区域。

该方法返回一个`Endpoint`对象。

了解更多关于[Vertex AI端点](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

In [ ]:
endpoint = aiplatform.Endpoint.create(
    display_name="production", project=PROJECT_ID, location=REGION
)

print(endpoint)

将`Model`资源部署到`Endpoint`资源

接下来，您将`Vertex AI Model`资源部署到一个`Vertex AI Endpoint`资源。 `Vertex AI Model`资源已经为其定义了部署容器镜像。 要部署，您需要指定以下额外的配置设置：

- 机器类型。
- (如果有的话) GPU的类型和数量。
- VM实例的静态、手动或自动缩放。

在此示例中，您将以指定参数的最小数目部署模型，如下所示：

- `model`：`Model`资源。
- `deployed_model_displayed_name`：部署模型实例的可读名称。
- `machine_type`：每个VM实例的机器类型。

由于需要为资源分配资源，这可能需要几分钟时间。

In [ ]:
response = endpoint.deploy(
    model=champion_model,
    deployed_model_display_name="champion",
    machine_type=DEPLOY_COMPUTE,
)

print(response)

## 为部署的模型创建多个竞争版本

接下来，您可以创建多个合成版本的部署冠军模型。

In [ ]:
contender_model_1 = aiplatform.Model.upload(
    display_name="resnet-v2",
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    parent_model=champion_model.resource_name,
    is_default_version=False,
    version_aliases=["v2"],
)

contender_model_2 = aiplatform.Model.upload(
    display_name="resnet-v3",
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    parent_model=champion_model.resource_name,
    is_default_version=False,
    version_aliases=["v3"],
)

接下来，为每个竞争模型创建一个模型评估。

In [ ]:
metrics = {"logLoss": 1.5, "auPrc": 0.83}

contender_1_eval = gapic.ModelEvaluation(
    display_name="train-v2",
    metrics_schema_uri="gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml",
    metrics=metrics,
)

# workaround for bug
contender_model_1.versioning_registry.add_version_aliases(
    new_aliases=["default"], version=contender_model_1.version_id
)

client.import_model_evaluation(
    parent=contender_model_1.resource_name, model_evaluation=contender_1_eval
)

metrics = {"logLoss": 1.6, "auPrc": 0.82}

contender_2_eval = gapic.ModelEvaluation(
    display_name="train-v3",
    metrics_schema_uri="gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml",
    metrics=metrics,
)

# workaround for bug
contender_model_2.versioning_registry.add_version_aliases(
    new_aliases=["default"], version=contender_model_2.version_id
)

client.import_model_evaluation(
    parent=contender_model_2.resource_name, model_evaluation=contender_2_eval
)

## 为管道创建自定义组件

接下来，您需要创建几个自定义组件，用于在管道中使用。

### 创建用于导入分类指标的组件

首先，您定义一个组件，用于将挑战模型的评估指标导入到模型注册表中。该组件接受以下参数：

- display_name：评估指标的可读名称
- metrics：格式化后的用于分类的评估指标
- parent_model_resource：挑战者模型版本的完整资源名称
- region：地区。

In [ ]:
@component(packages_to_install=["google-cloud-aiplatform"])
def import_classification_metrics(
    display_name: str,
    metrics: dict,
    parent_model_resource: str,
    project: str,
    region: str,
):
    from google.cloud import aiplatform
    from google.cloud.aiplatform import gapic

    print("DISPLAY", display_name)

    evaluation = gapic.ModelEvaluation(
        display_name=display_name,
        metrics_schema_uri="gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml",
        metrics=metrics,
    )

    # workaround for bug
    aiplatform.init(project=project, location=region)
    parent_model = aiplatform.Model(parent_model_resource)
    parent_model.versioning_registry.add_version_aliases(
        new_aliases=["default"], version=parent_model.version_id
    )

    API_ENDPOINT = f"{region}-aiplatform.googleapis.com"
    client = gapic.ModelServiceClient(client_options={"api_endpoint": API_ENDPOINT})
    client.import_model_evaluation(
        parent=parent_model_resource, model_evaluation=evaluation
    )

创建组件以比较指标

接下来，您需要定义一个组件来比较模型的冠军版本和挑战者版本之间的 `auPrc` 指标。拥有最好 `auPrc` 值的版本将被设为默认模型。在随后部署时，默认模型将被部署。该组件接受以下参数：

- `champion_resource_name`：冠军模型的完整资源名称。
- `contender_model_resource_names`：挑战者模型的完整资源名称列表。

In [ ]:
@component(packages_to_install=["google-cloud-aiplatform"])
def compare_metrics(
    champion_model_resource_name: str, contender_model_resource_names: list
):
    from google.cloud import aiplatform

    # Get the metrics for the blessed model
    champion_model = aiplatform.Model(champion_model_resource_name)
    champion_eval = champion_model.list_model_evaluations()[
        1
    ]  # index 1 is the production eval data
    champion_auPrc = champion_eval.metrics["auPrc"]

    champion_model.versioning_registry.add_version_aliases(
        new_aliases=["default"], version=champion_model.version_id
    )

    # Get the metrics for the challenger model
    for contender in contender_model_resource_names:
        contender_model = aiplatform.Model(contender)
        contender_eval = contender_model.list_model_evaluations()[1]
        contender_auPrc = contender_eval.metrics["auPrc"]

        # Which model has the best accuracy becomes the default model
        if contender_auPrc > champion_auPrc:
            contender_model.versioning_registry.add_version_aliases(
                new_aliases=["default"], version=contender_model.version_id
            )
            champion_auPrc = contender_auPrc

## 构建冠军对多竞争者管道

接下来，您需要为以下任务构建一个管道：

- 获取模型的冠军版本。
- 获取已部署冠军模型的端点。
- 使用生产数据重新训练和评估（伪造）冠军模型。
- 导入冠军模型在生产数据上的评估指标。
- 对于每个竞争者模型：
  - 获取模型的竞争者版本。
  - 使用生产数据重新训练和评估（伪造）竞争者模型。
- 比较冠军和竞争者的评估指标，并相应地设置默认模型。
- 导入现有的生产端点。
- 将新的默认模型部署到生产端点上。

In [ ]:
@kfp.dsl.pipeline(name="multicontender-vs-champion")
def pipeline(
    champion_model_resource: str,
    contender_model_resources: list,
    serving_container: str,
    machine_type: str,
    endpoint_resource_name: str,
    endpoint_resource_uri: str,
    project: str = PROJECT_ID,
    region: str = REGION,
):
    from google_cloud_pipeline_components.experimental.evaluation import \
        GetVertexModelOp
    from google_cloud_pipeline_components.types import artifact_types
    from google_cloud_pipeline_components.v1.endpoint import ModelDeployOp
    from kfp.v2.components import importer_node

    # Get the Vertex AI model resource of the blessed model
    champion = GetVertexModelOp(model_resource_name=champion_model_resource)

    # pretend to retrain and evaluate the champion with production data
    champion_metrics = {"logLoss": 1.3, "auPrc": 0.86}

    # upload the metrics for the champion version
    import_champion_metrics = import_classification_metrics(
        display_name="production",
        metrics=champion_metrics,
        parent_model_resource=champion_model_resource,
        project=project,
        region=region,
    ).after(champion)

    ix = 0
    with kfp.dsl.ParallelFor(contender_model_resources).after(
        import_champion_metrics
    ) as contender_model_resource:
        contender = GetVertexModelOp(model_resource_name=contender_model_resource)

        # pretend to retrain and evaluate the champion with production data
        contender_metrics = {"logLoss": 1.1, "auPrc": 0.88}

        ix += 1
        import_contender_metrics = import_classification_metrics(
            display_name=f"production_{ix}",
            metrics=contender_metrics,
            parent_model_resource=contender_model_resource,
            project=project,
            region=region,
        ).after(contender)

    # Select the best model
    compare = compare_metrics(
        champion_model_resource,
        contender_model_resources,
    ).after(import_contender_metrics)

    # import the production Endpoint
    endpoint = importer_node.importer(
        artifact_uri=endpoint_resource_uri,
        artifact_class=artifact_types.VertexEndpoint,
        metadata={"resourceName": endpoint_resource_name},
    )

    # deploy model to endpoint
    _ = ModelDeployOp(
        model=champion.outputs["model"],
        endpoint=endpoint.output,
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
        dedicated_resources_machine_type=machine_type,
        traffic_split={"0": 100},
    ).after(compare)

### 编译管道

接下来，您需要编译管道。

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="multicontender_vs_champion.json"
)

### 执行管道

最后，您执行管道，传递以下管道参数值：

- `blessed_model_resource`：模型当前受认可版本的完整资源名称。
- `serving_container`：用于部署模型的服务容器。
- `machine_type`：用于部署模型的机器类型。
- `endpoint_resource_name`：生产端点的完整资源名称。
- `endpoint_resource_uri`：生产端点的完整URI。
- `project`：项目ID。
- `region`：地区。

In [ ]:
PIPELINE_ROOT = "{}/pipeline_root/control".format(BUCKET_URI)

job = aiplatform.PipelineJob(
    display_name="multicontender_vs_champion",
    template_path="multicontender_vs_champion.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={
        "champion_model_resource": champion_model.resource_name,
        "contender_model_resources": [
            contender_model_1.resource_name,
            contender_model_2.resource_name,
        ],
        "serving_container": DEPLOY_IMAGE,
        "machine_type": DEPLOY_COMPUTE,
        "endpoint_resource_name": endpoint.resource_name,
        "endpoint_resource_uri": "https://us-central1-aiplatform.googleapis.com/v1/"
        + endpoint.resource_name,
        "project": PROJECT_ID,
        "region": REGION,
    },
    enable_caching=False,
)

job.run()

! rm multicontender_vs_champion.json

### 获取生产终端的最新状态

现在，流水线已经完成，竞争者（版本3）模型已经取代之前的冠军模型，部署到生产终端上。

接下来，您将展示生产终端部署模型的最新信息，然后展示流量分布情况。100%入口的资源ID是竞争者（版本3）模型的资源ID。

In [ ]:
gca_resource = endpoint.list(filter="display_name=production")[0].gca_resource
print("Deployed Models", gca_resource.deployed_models)
print("\n")
print("Traffic Split", gca_resource.traffic_split)

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
import os

# Delete endpoint resource
try:
    endpoint.undeploy_all()
    endpoint.delete()
except Exception as e:
    print(e)

# Delete model resource
try:
    champion_model.delete()
except Exception as e:
    print(e)

# Delete the pipeline resource
try:
    job.delete()
except Exception as e:
    print(e)

# Delete Cloud Storage objects that were created
delete_bucket = True
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI